In [ ]:
def setupSpark():
  # Spark needs to run with Java 8 ... 
  !pip install -q findspark
  !apt-get install openjdk-8-jdk-headless > /dev/null
  !echo 2 | update-alternatives --config java > /dev/null
  !java -version
  import os, findspark
  os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
  # !echo JAVA_HOME=$JAVA_HOME
  !pip install -q pyspark
  findspark.init(spark_home='/usr/local/lib/python3.7/dist-packages/pyspark')
  !pyspark --version


setupSpark()

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 59.5 MB/s 
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("accumulator").getOrCreate()

accum=spark.sparkContext.accumulator(0)
rdd=spark.sparkContext.parallelize([1,2,3,4,5,6,7,8,9])
rdd.foreach(lambda x:accum.add(x))
print(accum.value)

accuSum=spark.sparkContext.accumulator(0)
def countFun(x):
    global accuSum
    accuSum+=x 
rdd.foreach(countFun)
print(accuSum.value)

45
45


In [ ]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

states = {"KL":"Kerala", "TN":"Tamilnadu", "KA":"Karnataka"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("Simon","Hemal","USA","TN"),
    ("Shyam","Moses","USA","KL"),
    ("Sebin","James","USA","TN"),
    ("King","Maker","USA","KA")]

rdd = spark.sparkContext.parallelize(data)
columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show()
def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)


root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    Simon|   Hemal|    USA|   TN|
|    Shyam|   Moses|    USA|   KL|
|    Sebin|   James|    USA|   TN|
|     King|   Maker|    USA|   KA|
+---------+--------+-------+-----+

[('Simon', 'Hemal', 'USA', 'Tamilnadu'), ('Shyam', 'Moses', 'USA', 'Kerala'), ('Sebin', 'James', 'USA', 'Tamilnadu'), ('King', 'Maker', 'USA', 'Karnataka')]


In [ ]:
x = 50

def foo():
    x = 100
    print("local x:", x)


foo()
print("global x:", x)

local x: 100
global x: 50
